# FUNCTION DECLARATIONS

1. [getAspectDescription](#getAspectDescription)(text: string) => [{aspect: string, description: string}]
1. [getSentiment](#getSentiment)(raw_text: string) => (output: string, prediction: (polarity, subjectivity))

## getAspectDescription

In [14]:
# %pip install spacy

In [15]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [16]:
# getAspectDescription(text: string) => [{aspect: string, description: string}]

def getAspectDescription(text):
    """
    function getAspectDescription(text: string) -> [{aspect: string, description: string}]
    
    This function takes in a string of text and returns a list of dictionaries, each containing the aspect and its corresponding description.

    Args:
        text (str): The input text to analyze.

    Returns:
        list: A list of dictionaries, each containing the aspect and its corresponding description.

    Example:
        >>> text = "The camera is great but the battery life is short."
        >>> getAspectDescription(text)
        [{'aspect': ['camera'], 'description': 'great'}, {'aspect': ['battery', 'life'], 'description': 'short'}]
    """
    aspects = []

    doc = nlp(text)
    descriptive_term = ''
    target = []
    for token in doc:
        if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
            # target = token.text
            target.append(token.text)
        if token.pos_ == 'ADJ':
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            descriptive_term = f"{prepend}" + token.text

    aspects.append({'aspect': target, 'description': descriptive_term})

    return aspects


In [17]:
sentences = [
    "The sound quality is amazing and the earphones are very comfortable to wear. I would definitely recommend this product to anyone looking for a high-quality wireless earphone.",
    "These earphones are a great value for the price. They have a long battery life and the sound quality is really good. I'm very happy with my purchase.",
    "I'm really impressed with these earphones. They connect to my phone really quickly and the sound quality is excellent. Plus, they're very lightweight and comfortable to wear.",
    "These earphones are so convenient! I love that they're wireless and I don't have to worry about cords getting tangled. The sound quality is great too.",
    "I bought these earphones for working out and they're perfect. They stay in place really well and the sound quality is really good. I would definitely recommend them to anyone looking for wireless earphones for working out.",
    "These earphones are amazing! The sound quality is excellent and they're really comfortable to wear. Plus, the battery life is really impressive - I can use them for hours without having to recharge.",
    "I'm really happy with these earphones. They fit my ears really well and the sound quality is excellent. Plus, they're really easy to use - I can control the volume and playback from the earphones themselves.",
    "These earphones are so stylish! I love the sleek design and the sound quality is really good too. They're also really easy to pair with my phone.",
    "I'm very impressed with these earphones. They have a great range and I can use them all over my house without losing connection. The sound quality is also really good.",
    "These earphones are a great value for the price. They have a long battery life and the sound quality is really good. I would definitely recommend them to anyone looking for wireless earphones."
]

for sentence in sentences:
    print(getAspectDescription(sentence))

[{'aspect': ['product'], 'description': 'Poor'}]

## getSentiment

In [18]:
# %pip install pandas
# %pip install re
# %pip install bs4
# %pip install nltk
# %pip install pickle

In [19]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle

In [20]:
# getSentiment(raw_text: string) => (output: string, prediction: (polarity, subjectivity))

model = pickle.load(open('modelNB.pkl', 'rb'))

def getSentiment(raw_text: str):
    """
    function getSentiment(raw_text: string) -> (output: string, prediction: (polarity, subjectivity))

    This function takes in a string of raw text and performs sentiment analysis to determine whether the text is positive or negative. It returns a tuple consisting of the sentiment label and the positive probability of the prediction.

    Args:
        raw_text (str): The raw text to analyze.

    Returns:
        tuple: A tuple consisting of the sentiment label and the positive probability of the prediction.

    Example:
        >>> raw_text = "This product is amazing! I love it so much."
        >>> getSentiment(raw_text)
        ('Positive', 0.00819811, 0.99180189))
    """

    # Instantiate PorterStemmer
    p_stemmer = PorterStemmer()

    # Remove HTML
    review_text = BeautifulSoup(raw_text).get_text()

    # Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # Convert words to lower case and split each word up
    words = letters_only.lower().split()

    # Convert stopwords to a set
    stops = set(stopwords.words('english'))

    # Adding on stopwords that were appearing frequently in both positive and negative reviews
    stops.update(['app','shopee','shoppee','item','items','seller','sellers','bad'])

    # Remove stopwords
    meaningful_words = [w for w in words if w not in stops]

    # Stem words
    meaningful_words = [p_stemmer.stem(w) for w in meaningful_words]

    # Join words back into one string, with a space in between each word
    final_text = pd.Series(" ".join(meaningful_words))

    # Generate predictions
    pred = model.predict(final_text)[0]
    positive_prob = model.predict_proba([pd.Series.to_string(final_text)])[0][0]


    if pred == 1:
        output = "Negative"
    else:
        output = "Postive"

    return output, positive_prob


In [29]:
text = 'This phone sucks it doesnt even last long!'

In [22]:
output, proba = getSentiment(text)

In [23]:
print(f"Text: {text}", output, proba, sep='\t')

Text: The product is strong, but it was not the right color. Poor packaging	Negative	0.22424745443040556


# Aspect Sentiment Classification

In [24]:
sentences = [
    "The sound quality is amazing and the earphones are very comfortable to wear. I would definitely recommend this product to anyone looking for a high-quality wireless earphone.",
    "These earphones are a great value for the price. They have a long battery life and the sound quality is really good. I'm very happy with my purchase.",
    "I'm really impressed with these earphones. They connect to my phone really quickly and the sound quality is excellent. Plus, they're very lightweight and comfortable to wear.",
    "These earphones are so convenient! I love that they're wireless and I don't have to worry about cords getting tangled. The sound quality is great too.",
    "I bought these earphones for working out and they're perfect. They stay in place really well and the sound quality is really good. I would definitely recommend them to anyone looking for wireless earphones for working out.",
    "These earphones are amazing! The sound quality is excellent and they're really comfortable to wear. Plus, the battery life is really impressive - I can use them for hours without having to recharge.",
    "I'm really happy with these earphones. They fit my ears really well and the sound quality is excellent. Plus, they're really easy to use - I can control the volume and playback from the earphones themselves.",
    "These earphones are so stylish! I love the sleek design and the sound quality is really good too. They're also really easy to pair with my phone.",
    "I'm very impressed with these earphones. They have a great range and I can use them all over my house without losing connection. The sound quality is also really good.",
    "These earphones are a great value for the price. They have a long battery life and the sound quality is really good. I would definitely recommend them to anyone looking for wireless earphones."
]

aspectSentiments = []
for sentence in sentences:
    aspects = getAspectDescription(sentence)
    aspectSentiment = [{'aspect': aspect['aspect'], 'description': aspect['description'], 'sentiment': getSentiment(sentence)} for aspect in aspects]
    aspectSentiments.extend(aspectSentiment)

for item in aspectSentiments:
    print(f"Aspect: {item['aspect']}\nDescription: {item['description']}\nSentiment: {item['sentiment']}\n")

[{'aspect': ['food'], 'description': 'delicious', 'sentiment': ('Postive', 0.6099928109273904)}]


# Aspect-Sentence Mapping

In [9]:
sentences = [
    "The phone has a huge display that is crystal clear. I love watching videos on it.",
    "This phone is compact and fits perfectly in my pocket. The size is just right.",
    "The camera on this phone is amazing. The pictures come out clear and vivid.",
    "The battery life on this phone is incredible. I can go all day without needing to charge it.",
    "I'm impressed with the phone's sleek design. It looks great and feels good in my hand.",
    "The phone's display is one of the best I've seen. The colors are vibrant and the screen is bright.",
    "The size of this phone is perfect for one-handed use. It's not too big or too small.",
    "The camera on this phone takes amazing low light photos. I'm really happy with the quality.",
    "The battery on this phone lasts all day and charges quickly. I'm never without a charged phone.",
    "I'm in love with the phone's display. It's so clear and bright, even in direct sunlight."
]

#keys = []

#for sentence in sentences:
    #aspects = getAspectDescription(sentence)
    #for aspect in aspects:
        #keys.append(str(aspect['aspect']))
#aspect_dict = {key: [] for key in keys}

#for item in aspectSentiments:
    #sentiment = item['sentiment']
    #for aspect in aspects:
        #if aspect not in aspect
    
#for key in keys:
    #print(key)


aspect_dict = {}
for sentence in sentences:
    aspects = getAspectDescription(sentence)
    for aspect in aspects:
        aspect_name = tuple(aspect['aspect'])
        for item in aspectSentiments:
            sentiment = item['sentiment']
        if aspect_name not in aspect_dict:
            aspect_dict[aspect_name] = {}
        aspect_dict[aspect_name][sentence] = sentiment

for aspect, sentences in aspect_dict.items():
    #num_sentences = len(sentences)
    #total_sentiment = sum(sentences.values())
    print(f"{aspect}:")
    for sentence, sentiment in sentences.items():
        print(f" - {sentence}: {sentiment}")


('phone',):
 - The phone has a huge display that is crystal clear. I love watching videos on it.: Postive
('phone', 'size'):
 - This phone is compact and fits perfectly in my pocket. The size is just right.: Postive
('camera', 'pictures'):
 - The camera on this phone is amazing. The pictures come out clear and vivid.: Postive
('life',):
 - The battery life on this phone is incredible. I can go all day without needing to charge it.: Postive
():
 - I'm impressed with the phone's sleek design. It looks great and feels good in my hand.: Postive
 - The battery on this phone lasts all day and charges quickly. I'm never without a charged phone.: Postive
 - I'm in love with the phone's display. It's so clear and bright, even in direct sunlight.: Postive
('display', 'colors', 'screen'):
 - The phone's display is one of the best I've seen. The colors are vibrant and the screen is bright.: Postive
('size',):
 - The size of this phone is perfect for one-handed use. It's not too big or too small.: 

Craig

In [27]:
sentences = [
    "The phone has a huge display that is crystal clear. I love watching videos on it.",
    "This phone is compact and fits perfectly in my pocket. The size is just right.",
    "The camera on this phone is amazing. The pictures come out clear and vivid.",
    "The battery life on this phone is incredible. I can go all day without needing to charge it.",
    "I'm impressed with the phone's sleek design. It looks great and feels good in my hand.",
    "The phone's display is one of the best I've seen. The colors are vibrant and the screen is bright.",
    "The size of this phone is perfect for one-handed use. It's not too big or too small.",
    "The camera on this phone takes amazing low light photos. I'm really happy with the quality.",
    "The battery on this phone lasts all day and charges quickly. I'm never without a charged phone.",
    "I'm in love with the phone's display. It's so clear and bright, even in direct sunlight."
]

#keys = []

#for sentence in sentences:
    #aspects = getAspectDescription(sentence)
    #for aspect in aspects:
        #keys.append(str(aspect['aspect']))
#aspect_dict = {key: [] for key in keys}

#for item in aspectSentiments:
    #sentiment = item['sentiment']
    #for aspect in aspects:
        #if aspect not in aspect
    
#for key in keys:
    #print(key)


aspect_dict = {}
for sentence in sentences:
    aspects = getAspectDescription(sentence)
    for aspect in aspects:
        aspect_name = tuple(aspect['aspect'])
        for item in aspectSentiment:
            sentiment = item['sentiment']
        if aspect_name not in aspect_dict:
            aspect_dict[aspect_name] = {}
        aspect_dict[aspect_name][sentence] = sentiment

for aspect, sentences in aspect_dict.items():
    print(f"{aspect}:")
    for sentence, sentiment in sentences.items():
        print(f" - {sentence}: {sentiment}")

('phone',):
 - The phone has a huge display that is crystal clear. I love watching videos on it.: ('Postive', 0.6099928109273904)
('phone', 'size'):
 - This phone is compact and fits perfectly in my pocket. The size is just right.: ('Postive', 0.6099928109273904)
('camera', 'pictures'):
 - The camera on this phone is amazing. The pictures come out clear and vivid.: ('Postive', 0.6099928109273904)
('life',):
 - The battery life on this phone is incredible. I can go all day without needing to charge it.: ('Postive', 0.6099928109273904)
():
 - I'm impressed with the phone's sleek design. It looks great and feels good in my hand.: ('Postive', 0.6099928109273904)
 - The battery on this phone lasts all day and charges quickly. I'm never without a charged phone.: ('Postive', 0.6099928109273904)
 - I'm in love with the phone's display. It's so clear and bright, even in direct sunlight.: ('Postive', 0.6099928109273904)
('display', 'colors', 'screen'):
 - The phone's display is one of the best I